In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd

# Passo 1: Puxar as tabelas 

In [2]:

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = 'https://www.etf.com/etfanalytics/etf-finder'

driver.get(url)

time.sleep(5)

cem_resultados = driver.find_element("xpath", 
                    '''/html/body/div[5]/section/div/div[3]/section/div/
                    div/div/div/div[2]/section[2]/div[2]/section[2]/div[1]/div/div[4]/button/label/span''')

driver.execute_script("arguments[0].click();", cem_resultados)

time.sleep(2)

numero_paginas = driver.find_element("xpath", '''//*[@id="totalPages"]''')

numero_paginas = numero_paginas.text.replace("of ", "")

numero_paginas = int(numero_paginas)

lista_data_frames_por_pagina = []

for pagina in range(0, numero_paginas):

    local_tabela = '''
    //*[@id="finderTable"]
    '''

    elemento = driver.find_element("xpath", local_tabela)

    html_tabela = elemento.get_attribute('outerHTML')

    tabela = pd.read_html(html_tabela)[0]
    
    lista_data_frames_por_pagina.append(tabela)
    
    if pagina != numero_paginas:
    
        botao_avancar_pag = driver.find_element("xpath", '//*[@id="nextPage"]')

        driver.execute_script("arguments[0].click();", botao_avancar_pag)


    

base_dados_etf_cadastro = pd.concat(lista_data_frames_por_pagina)

#voltando tudo

for pagina in range(0, numero_paginas):

    botao_voltar_pag = driver.find_element("xpath", '//*[@id="previousPage"]')

    driver.execute_script("arguments[0].click();", botao_voltar_pag)

# a unica coisa que muda é esse botão
botao_dados_perf = driver.find_element("xpath", '/html/body/div[5]/section/div/div[3]/section/div/div/div/div/div[2]/section[2]/div[2]/ul/li[2]/span')


# a partir daqui é tudo igual
driver.execute_script("arguments[0].click();", botao_dados_perf)

lista_data_frames_por_pagina = []

for pagina in range(0, numero_paginas):


    local_tabela = '''
    //*[@id="finderTable"]
    '''

    elemento = driver.find_element("xpath", local_tabela)

    html_tabela = elemento.get_attribute('outerHTML')

    tabela = pd.read_html(html_tabela)[0]
    
    lista_data_frames_por_pagina.append(tabela)
    
    if pagina != numero_paginas:
    
        botao_avancar_pag = driver.find_element("xpath", '//*[@id="nextPage"]')

        driver.execute_script("arguments[0].click();", botao_avancar_pag)
    

base_dados_etf_performance = pd.concat(lista_data_frames_por_pagina)

#fechando navegador
driver.quit()



In [3]:
base_dados_etf_cadastro = base_dados_etf_cadastro.set_index('Ticker')
base_dados_etf_performance = base_dados_etf_performance.set_index("Ticker")


In [4]:
base_dados_etf_cadastro

,Name,Segment,Issuer,Expense Ratio,AUM
Ticker,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$379.46B
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$311.66B
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$285.77B
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$284.73B
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$163.27B
...,...,...,...,...,...
CDEI,"Calvert US Large-Cap Diversity, Equity and Inc...",Equity: U.S. - Large Cap,Morgan Stanley,0.14%,--
CVSE,Calvert US Select Equity ETF,Equity: U.S. - Large Cap,Morgan Stanley,0.29%,--
SROI,Calamos Antetokounmpo Global Sustainable Equit...,Equity: Global - Total Market,Calamos Family Partners,--,--


In [5]:
base_dados_etf_performance

,Name,1 Month,3 Month,YTD,1 Year,3 Years,5 Years,10 Years,As Of Date
Ticker,,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,3.88%,4.37%,8.25%,-5.69%,8.76%,10.57%,12.62%,02/15/23
IVV,iShares Core S&P 500 ETF,3.87%,4.37%,8.24%,-5.68%,8.77%,10.61%,12.66%,02/15/23
VTI,Vanguard Total Stock Market ETF,4.21%,4.55%,9.03%,-6.27%,8.34%,10.13%,12.26%,02/15/23
VOO,Vanguard S&P 500 ETF,3.89%,4.39%,8.29%,-5.68%,8.75%,10.60%,12.67%,02/15/23
QQQ,Invesco QQQ Trust,10.01%,7.07%,16.08%,-12.57%,10.31%,14.10%,17.49%,02/15/23
...,...,...,...,...,...,...,...,...,...
CDEI,"Calvert US Large-Cap Diversity, Equity and Inc...",--,--,--,--,--,--,--,02/15/23
CVSE,Calvert US Select Equity ETF,--,--,--,--,--,--,--,02/15/23
SROI,Calamos Antetokounmpo Global Sustainable Equit...,--,--,--,--,--,--,--,02/15/23


# Passo 2: Escolher quais períodos de rentabilidade analisar 

In [6]:
base_dados_etf_performance = base_dados_etf_performance[['3 Years', '5 Years', '10 Years']]

base_dados_etf_performance

,3 Years,5 Years,10 Years
Ticker,,,
SPY,8.76%,10.57%,12.62%
IVV,8.77%,10.61%,12.66%
VTI,8.34%,10.13%,12.26%
VOO,8.75%,10.60%,12.67%
QQQ,10.31%,14.10%,17.49%
...,...,...,...
CDEI,--,--,--
CVSE,--,--,--
SROI,--,--,--


# Passo 3: Retirar dados faltantes. Fundos que não possuem os períodos necessários 

In [7]:
base_dados_etf_performance = base_dados_etf_performance.replace("--", pd.NA)

base_dados_etf_performance

,3 Years,5 Years,10 Years
Ticker,,,
SPY,8.76%,10.57%,12.62%
IVV,8.77%,10.61%,12.66%
VTI,8.34%,10.13%,12.26%
VOO,8.75%,10.60%,12.67%
QQQ,10.31%,14.10%,17.49%
...,...,...,...
CDEI,NaN,NaN,NaN
CVSE,NaN,NaN,NaN
SROI,NaN,NaN,NaN


In [8]:
base_dados_etf_performance = base_dados_etf_performance.dropna()

base_dados_etf_performance



,3 Years,5 Years,10 Years
Ticker,,,
SPY,8.76%,10.57%,12.62%
IVV,8.77%,10.61%,12.66%
VTI,8.34%,10.13%,12.26%
VOO,8.75%,10.60%,12.67%
QQQ,10.31%,14.10%,17.49%
...,...,...,...
SZK,-29.51%,-24.56%,-23.80%
LD,3.92%,-4.25%,-2.76%
ERUS,-40.34%,-23.14%,-12.89%


# Transformar as rentabilidades em números decimais 

In [9]:
base_dados_etf_performance['3 Years'] = (base_dados_etf_performance['3 Years'].
                                         str.rstrip('%').astype(float)/100)

base_dados_etf_performance['5 Years'] = (base_dados_etf_performance['5 Years'].
                                         str.rstrip('%').astype(float)/100)

base_dados_etf_performance['10 Years'] = (base_dados_etf_performance['10 Years'].
                                         str.rstrip('%').astype(float)/100)

base_dados_etf_performance 

/var/folders/5f/tzy6_1yx2c51xslpfdfhbzw40000gn/T/ipykernel_18636/2916189344.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_dados_etf_performance['3 Years'] = (base_dados_etf_performance['3 Years'].
/var/folders/5f/tzy6_1yx2c51xslpfdfhbzw40000gn/T/ipykernel_18636/2916189344.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_dados_etf_performance['5 Years'] = (base_dados_etf_performance['5 Years'].
/var/folders/5f/tzy6_1yx2c51xslpfdfhbzw40000gn/T/ipykernel_18636/2916189344.py:7: SettingWithCop

,3 Years,5 Years,10 Years
Ticker,,,
SPY,0.0876,0.1057,0.1262
IVV,0.0877,0.1061,0.1266
VTI,0.0834,0.1013,0.1226
VOO,0.0875,0.1060,0.1267
QQQ,0.1031,0.1410,0.1749
...,...,...,...
SZK,-0.2951,-0.2456,-0.2380
LD,0.0392,-0.0425,-0.0276
ERUS,-0.4034,-0.2314,-0.1289


# Juntar as duas tabelas - Dados cadastrados e Performace 
    # how = inner é para juntar as duas tabelas com o mesmo dado 

In [10]:
base_dados_final = base_dados_etf_cadastro.join(base_dados_etf_performance, how = "inner")

base_dados_final

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$379.46B,0.0876,0.1057,0.1262
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$311.66B,0.0877,0.1061,0.1266
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$285.77B,0.0834,0.1013,0.1226
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$284.73B,0.0875,0.1060,0.1267
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$163.27B,0.1031,0.1410,0.1749
...,...,...,...,...,...,...,...,...
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.28M,-0.2951,-0.2456,-0.2380
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.22M,0.0392,-0.0425,-0.0276
ERUS,iShares MSCI Russia ETF,Equity: Russia - Total Market,Blackrock,0.08%,$961.13K,-0.4034,-0.2314,-0.1289


# Passo 6 filtrar os ETFs alavancados

In [11]:
base_dados_final = base_dados_final[~base_dados_final['Segment'].str.contains("Leveraged")]

base_dados_final

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years
Ticker,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$379.46B,0.0876,0.1057,0.1262
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$311.66B,0.0877,0.1061,0.1266
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$285.77B,0.0834,0.1013,0.1226
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$284.73B,0.0875,0.1060,0.1267
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$163.27B,0.1031,0.1410,0.1749
...,...,...,...,...,...,...,...,...
RXD,ProShares UltraShort Health Care,Inverse Equity: U.S. Health Care,ProShares,0.95%,$1.91M,-0.2708,-0.2600,-0.2862
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.28M,-0.2951,-0.2456,-0.2380
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.22M,0.0392,-0.0425,-0.0276


# Passo numero 7: Ranking de retabilidade por período e soma-los 

In [12]:
base_dados_final['rank_3_anos'] = base_dados_final['3 Years'].rank(ascending = False)

base_dados_final['rank_5_anos'] = base_dados_final['5 Years'].rank(ascending = False)

base_dados_final['rank_10_anos'] = base_dados_final['10 Years'].rank(ascending = False)

base_dados_final

/var/folders/5f/tzy6_1yx2c51xslpfdfhbzw40000gn/T/ipykernel_18636/4009424343.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_dados_final['rank_3_anos'] = base_dados_final['3 Years'].rank(ascending = False)
/var/folders/5f/tzy6_1yx2c51xslpfdfhbzw40000gn/T/ipykernel_18636/4009424343.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_dados_final['rank_5_anos'] = base_dados_final['5 Years'].rank(ascending = False)
/var/folders/5f/tzy6_1yx2c51xslpfdfhbzw40000gn/T/ipykernel_18636/4009424343.py:5: Se

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years,rank_3_anos,rank_5_anos,rank_10_anos
Ticker,,,,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$379.46B,0.0876,0.1057,0.1262,214.0,97.0,78.0
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$311.66B,0.0877,0.1061,0.1266,212.0,93.5,75.0
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$285.77B,0.0834,0.1013,0.1226,243.0,114.5,98.0
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$284.73B,0.0875,0.1060,0.1267,215.0,95.0,73.5
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$163.27B,0.1031,0.1410,0.1749,151.0,20.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...
RXD,ProShares UltraShort Health Care,Inverse Equity: U.S. Health Care,ProShares,0.95%,$1.91M,-0.2708,-0.2600,-0.2862,806.0,811.0,820.0
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.28M,-0.2951,-0.2456,-0.2380,810.0,809.0,809.0
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.22M,0.0392,-0.0425,-0.0276,422.0,742.0,722.5


In [13]:
base_dados_final['rank_final'] = (base_dados_final['rank_3_anos']+
                                 base_dados_final['rank_5_anos']+
                                 base_dados_final['rank_10_anos'])

base_dados_final

/var/folders/5f/tzy6_1yx2c51xslpfdfhbzw40000gn/T/ipykernel_18636/684574445.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_dados_final['rank_final'] = (base_dados_final['rank_3_anos']+


,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years,rank_3_anos,rank_5_anos,rank_10_anos,rank_final
Ticker,,,,,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$379.46B,0.0876,0.1057,0.1262,214.0,97.0,78.0,389.0
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$311.66B,0.0877,0.1061,0.1266,212.0,93.5,75.0,380.5
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$285.77B,0.0834,0.1013,0.1226,243.0,114.5,98.0,455.5
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$284.73B,0.0875,0.1060,0.1267,215.0,95.0,73.5,383.5
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$163.27B,0.1031,0.1410,0.1749,151.0,20.0,10.0,181.0
...,...,...,...,...,...,...,...,...,...,...,...,...
RXD,ProShares UltraShort Health Care,Inverse Equity: U.S. Health Care,ProShares,0.95%,$1.91M,-0.2708,-0.2600,-0.2862,806.0,811.0,820.0,2437.0
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.28M,-0.2951,-0.2456,-0.2380,810.0,809.0,809.0,2428.0
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.22M,0.0392,-0.0425,-0.0276,422.0,742.0,722.5,1886.5


In [14]:
base_dados_final["3 Years"] = base_dados_final["3 Years"].apply(lambda x: str(round(x * 100, 2)) + "%")

base_dados_final["5 Years"] = base_dados_final["5 Years"].apply(lambda x: str(round(x * 100, 2)) + "%")

base_dados_final["10 Years"] = base_dados_final["10 Years"].apply(lambda x:str(round(x * 100, 2)) + "%")

base_dados_final


/var/folders/5f/tzy6_1yx2c51xslpfdfhbzw40000gn/T/ipykernel_18636/4277560533.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_dados_final["3 Years"] = base_dados_final["3 Years"].apply(lambda x: str(round(x * 100, 2)) + "%")
/var/folders/5f/tzy6_1yx2c51xslpfdfhbzw40000gn/T/ipykernel_18636/4277560533.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  base_dados_final["5 Years"] = base_dados_final["5 Years"].apply(lambda x: str(round(x * 100, 2)) + "%")
/var/folders/5f/tzy6_1yx2c51xslpfdfhbzw40000gn/T

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years,rank_3_anos,rank_5_anos,rank_10_anos,rank_final
Ticker,,,,,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$379.46B,8.76%,10.57%,12.62%,214.0,97.0,78.0,389.0
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$311.66B,8.77%,10.61%,12.66%,212.0,93.5,75.0,380.5
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$285.77B,8.34%,10.13%,12.26%,243.0,114.5,98.0,455.5
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$284.73B,8.75%,10.6%,12.67%,215.0,95.0,73.5,383.5
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$163.27B,10.31%,14.1%,17.49%,151.0,20.0,10.0,181.0
...,...,...,...,...,...,...,...,...,...,...,...,...
RXD,ProShares UltraShort Health Care,Inverse Equity: U.S. Health Care,ProShares,0.95%,$1.91M,-27.08%,-26.0%,-28.62%,806.0,811.0,820.0,2437.0
SZK,ProShares UltraShort Consumer Goods,Inverse Equity: U.S. Consumer Staples,ProShares,0.95%,$1.28M,-29.51%,-24.56%,-23.8%,810.0,809.0,809.0,2428.0
LD,iPath Bloomberg Lead Subindex Total Return ETN,Commodities: Industrial Metals Lead,Barclays Capital Inc.,0.70%,$1.22M,3.92%,-4.25%,-2.76%,422.0,742.0,722.5,1886.5


# Passo 8: Ordenar do menor para o maior encontrando os melhores ETF

In [15]:
melhores_etfs = base_dados_final.sort_values(by = "rank_final")

melhores_etfs

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years,rank_3_anos,rank_5_anos,rank_10_anos,rank_final
Ticker,,,,,,,,,,,,
QCLN,First Trust NASDAQ Clean Edge Green Energy Ind...,Equity: U.S. Renewable Energy,First Trust,0.58%,$1.83B,26.5%,24.46%,18.86%,12.0,3.0,6.0,21.0
XSD,SPDR S&P Semiconductor ETF,Equity: U.S. Semiconductors,State Street Global Advisors,0.35%,$1.37B,24.35%,25.31%,24.38%,19.0,2.0,1.0,22.0
TAN,Invesco Solar ETF,Equity: Global Renewable Energy,Invesco,0.69%,$2.42B,28.2%,26.55%,16.24%,11.0,1.0,17.5,29.5
SMH,VanEck Semiconductor ETF,Equity: Global Semiconductors,VanEck,0.35%,$7.86B,19.77%,21.08%,23.28%,30.0,4.0,4.0,38.0
PSI,Invesco Dynamic Semiconductors ETF,Equity: U.S. Semiconductors,Invesco,0.56%,$576.98M,19.43%,19.46%,23.83%,32.0,6.0,2.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...
FAZ,Direxion Daily Financial Bear 3X Shares,Inverse Equity: U.S. Financials,Direxion,1.01%,$168.66M,-56.69%,-48.18%,-45.94%,834.0,834.0,833.0,2501.0
DUST,Direxion Daily Gold Miners Index Bear 2x Shares,Inverse Equity: Global Gold Miners,Direxion,0.92%,$116.02M,-54.39%,-52.16%,-53.25%,829.0,836.0,837.0,2502.0
SSG,ProShares UltraShort Semiconductors,Inverse Equity: U.S. Semiconductors,ProShares,0.95%,$6.76M,-57.63%,-51.74%,-48.23%,835.0,835.0,835.0,2505.0


In [16]:
melhores_etfs.head(10)

,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years,rank_3_anos,rank_5_anos,rank_10_anos,rank_final
Ticker,,,,,,,,,,,,
QCLN,First Trust NASDAQ Clean Edge Green Energy Ind...,Equity: U.S. Renewable Energy,First Trust,0.58%,$1.83B,26.5%,24.46%,18.86%,12.0,3.0,6.0,21.0
XSD,SPDR S&P Semiconductor ETF,Equity: U.S. Semiconductors,State Street Global Advisors,0.35%,$1.37B,24.35%,25.31%,24.38%,19.0,2.0,1.0,22.0
TAN,Invesco Solar ETF,Equity: Global Renewable Energy,Invesco,0.69%,$2.42B,28.2%,26.55%,16.24%,11.0,1.0,17.5,29.5
SMH,VanEck Semiconductor ETF,Equity: Global Semiconductors,VanEck,0.35%,$7.86B,19.77%,21.08%,23.28%,30.0,4.0,4.0,38.0
PSI,Invesco Dynamic Semiconductors ETF,Equity: U.S. Semiconductors,Invesco,0.56%,$576.98M,19.43%,19.46%,23.83%,32.0,6.0,2.0,40.0
SOXX,iShares Semiconductor ETF,Equity: U.S. Semiconductors,Blackrock,0.35%,$7.40B,18.89%,20.66%,23.73%,36.0,5.0,3.0,44.0
PTF,Invesco DWA Technology Momentum ETF,Equity: U.S. Information Technology,Invesco,0.60%,$222.73M,14.37%,18.1%,16.55%,71.0,7.0,15.0,93.0
XLK,Technology Select Sector SPDR Fund,Equity: U.S. Information Technology,State Street Global Advisors,0.10%,$42.61B,13.32%,17.8%,18.82%,84.0,9.0,7.0,100.0
RWJ,Invesco S&P SmallCap 600 Revenue ETF,Equity: U.S. - Small Cap,Invesco,0.39%,$1.20B,26.01%,13.67%,12.92%,14.0,23.5,65.5,103.0


In [17]:
base_dados_final.head(50)



,Name,Segment,Issuer,Expense Ratio,AUM,3 Years,5 Years,10 Years,rank_3_anos,rank_5_anos,rank_10_anos,rank_final
Ticker,,,,,,,,,,,,
SPY,SPDR S&P 500 ETF Trust,Equity: U.S. - Large Cap,State Street Global Advisors,0.09%,$379.46B,8.76%,10.57%,12.62%,214.0,97.0,78.0,389.0
IVV,iShares Core S&P 500 ETF,Equity: U.S. - Large Cap,Blackrock,0.03%,$311.66B,8.77%,10.61%,12.66%,212.0,93.5,75.0,380.5
VTI,Vanguard Total Stock Market ETF,Equity: U.S. - Total Market,Vanguard,0.03%,$285.77B,8.34%,10.13%,12.26%,243.0,114.5,98.0,455.5
VOO,Vanguard S&P 500 ETF,Equity: U.S. - Large Cap,Vanguard,0.03%,$284.73B,8.75%,10.6%,12.67%,215.0,95.0,73.5,383.5
QQQ,Invesco QQQ Trust,Equity: U.S. - Large Cap,Invesco,0.20%,$163.27B,10.31%,14.1%,17.49%,151.0,20.0,10.0,181.0
VEA,Vanguard FTSE Developed Markets ETF,Equity: Developed Markets Ex-U.S. - Total Market,Vanguard,0.05%,$109.24B,4.0%,3.08%,5.38%,418.5,455.0,390.0,1263.5
VTV,Vanguard Value ETF,Equity: U.S. - Large Cap Value,Vanguard,0.04%,$105.56B,8.87%,8.9%,11.35%,202.5,188.0,146.5,537.0
IEFA,iShares Core MSCI EAFE ETF,Equity: Developed Markets Ex-North America - T...,Blackrock,0.07%,$96.33B,3.6%,2.85%,5.35%,435.0,470.0,391.5,1296.5
BND,Vanguard Total Bond Market ETF,"Fixed Income: U.S. - Broad Market, Broad-based...",Vanguard,0.03%,$87.36B,-2.95%,0.72%,1.21%,689.5,598.0,602.0,1889.5
